In [1]:
# 使用GPTQ量化

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
     bits=4,
     group_size=128,
     dataset="ptb",
     desc_act=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')
quant_model = AutoModelForCausalLM.from_pretrained(model_id,device_map='auto')


/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)

text = "Hello, nice to see you"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Hello, nice to see you here. I'm a newbie here, but I'm trying to learn as much as I can. I'm a bit confused about the whole "noob" thing. I'm not sure if I'm supposed to be a noob or not. I'm not sure if I'm supposed to be a noob


In [1]:
# 使用AWQ量化
from transformers import  AutoTokenizer
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from transformers import AwqConfig, AutoConfig

quant_path = "models/opt-27b-awq"
model_id = "facebook/opt-2.7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoAWQForCausalLM.from_pretrained(model_id, device_map='cuda')

quantization_config = AwqConfig(
    bits=4,
    group_size=128,
    zero_point=True,
    version='gemm',
).to_dict()

model.model.config.quantization = quantization_config
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)


/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|███| 9/9 [00:00<00:00, 59167.30it/s]
/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacty of 11.70 GiB of which 64.56 MiB is free. Including non-PyTorch memory, this process has 9.63 GiB memory in use. Of the allocated memory 9.10 GiB is allocated by PyTorch, and 358.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF